In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [6]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__10clips10prims.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__10clips100prims-enc_config.yml'

In [7]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__10clips100prims'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [8]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [9]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)

In [10]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__10clips100prims/checkpoints/ckpt-999'

In [14]:
model.blend_vec_to_seq(np.eye(enc_config_yaml['num_primitives']))

<tf.Tensor: shape=(100, 120, 147), dtype=float32, numpy=
array([[[ 7.52901938e-03,  7.52930110e-03,  7.52972485e-03, ...,
          7.52927503e-03,  7.52935093e-03,  7.52948178e-03],
        [ 7.52897095e-03,  7.52930297e-03,  7.52979657e-03, ...,
          7.52938306e-03,  7.52941472e-03,  7.52936536e-03],
        [ 7.52906641e-03,  7.52924336e-03,  7.52974208e-03, ...,
          7.52932392e-03,  7.52946781e-03,  7.52921496e-03],
        ...,
        [ 7.52925407e-03,  7.52929272e-03,  7.52957631e-03, ...,
          7.52932485e-03,  7.52922613e-03,  7.52937514e-03],
        [ 7.52930203e-03,  7.52934441e-03,  7.52943847e-03, ...,
          7.52934488e-03,  7.52935046e-03,  7.52912788e-03],
        [ 7.52918469e-03,  7.52918469e-03,  7.52963871e-03, ...,
          7.52941333e-03,  7.52930949e-03,  7.52937933e-03]],

       [[ 7.52923265e-03,  7.52923265e-03,  7.52923265e-03, ...,
          7.52923265e-03,  7.52923265e-03,  7.52923265e-03],
        [ 7.52923265e-03,  7.52923265e-03,  7.